In [ ]:
!rm -rf /content/dataset/.ipynb_checkpoints
!rm -rf /content/dataset/train/.ipynb_checkpoints
!rm -rf /content/dataset/val/.ipynb_checkpoints
!rm -rf /content/dataset/train/positive/.ipynb_checkpoints
!rm -rf /content/dataset/train/negative/.ipynb_checkpoints
!rm -rf /content/dataset/val/negative/.ipynb_checkpoints
!rm -rf /content/dataset/val/positive/.ipynb_checkpoints


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import WeightedRandomSampler
import numpy as np
import random

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(45)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [ ]:
# @title
train_transforms = transforms.Compose([
    transforms.Resize(256),

    transforms.RandomResizedCrop(
        224,
        scale=(0.6, 1.0),   # koliko smije biti izrezano
        ratio=(0.75, 1.33)
    ),

    transforms.RandomHorizontalFlip(p=0.5),

    transforms.RandomRotation(degrees=15),

    transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.05
    ),

    transforms.ToTensor(),

    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])



In [ ]:
train_dataset = datasets.ImageFolder(
    root="/content/dataset/train",
    transform=train_transforms
)

val_dataset = datasets.ImageFolder(
    root="/content/dataset/val",
    transform=val_transforms
)

targets = train_dataset.targets
class_counts = np.bincount(targets)

class_weights = 1. / class_counts
sample_weights = class_weights[targets]

sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    sampler=sampler
)

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class_names = train_dataset.classes
print(class_names)


['negative', 'positive']


In [ ]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 128MB/s]


In [ ]:
for param in model.parameters():
    param.requires_grad = False


In [ ]:
num_features = model.fc.in_features

model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(model.fc.in_features, 2)
) # 2 nove klase


model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(
    model.fc.parameters(),  # trenira se SAMO novi sloj
    lr=0.001
)


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / len(loader), correct / total


In [ ]:
def evaluate(model, loader, criterion):
    model.eval()
    loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss += criterion(outputs, labels).item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return loss / len(loader), correct / total


In [ ]:
epochs = 30

for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion
    )
    val_loss, val_acc = evaluate(
        model, val_loader, criterion
    )

    print(f"Epoch {epoch+1}/{epochs}")
    print(f" Train loss: {train_loss:.4f}, acc: {train_acc:.4f}")
    print(f" Val   loss: {val_loss:.4f}, acc: {val_acc:.4f}")


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "resnet_30ep_mk2.pth")